In [3]:
import importlib
import dataset
importlib.reload(dataset)
from dataset import extract_matrix_users_movies_ratings, extract_users_movies_ratings_lists

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import random
from dataset import extract_matrix_users_movies_ratings, extract_users_movies_ratings_lists
from libreco.data import split_by_ratio_chrono, DatasetPure
from libreco.algorithms import ALS
from sklearn.metrics import mean_squared_error

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
def save_predictions_from_pandas(res_path, predictions, index_pd):
    index_pd = index_pd.astype({'Prediction': 'float'})
    index_pd.iloc[:, 1] = predictions
    index_pd.to_csv(res_path, index=False, float_format='%.3f')

In [5]:
def save_predictions(res_path, predictions):
    test_pd = pd.read_csv('../data/sampleSubmission.csv')
    test_pd = test_pd.astype({'Prediction': 'float'})
    test_pd.iloc[:, 1] = predictions
    test_pd.to_csv(res_path, index=False, float_format='%.3f')

In [8]:
RANDOM_STATE = 58

scores = []
test_pd = pd.read_csv('../data/sampleSubmission.csv')
users_test, movies_test, _ = extract_users_movies_ratings_lists(test_pd)

In [5]:
for i in range(5):
    train_pd = pd.read_csv(f'../data_val_train_kfold/partition_{i}_train.csv')
    val_pd = pd.read_csv(f'../data_val_train_kfold/partition_{i}_val.csv')
    users_train, movies_train, ratings_train = extract_users_movies_ratings_lists(train_pd)
    users_val, movies_val, ratings_val = extract_users_movies_ratings_lists(val_pd)
    train = pd.DataFrame({'user': users_train, 'item': movies_train, 'label': ratings_train})
    val = pd.DataFrame({'user': users_val, 'item': movies_val, 'label': ratings_val})
    train_data, data_info = DatasetPure.build_trainset(train)
    eval_data = DatasetPure.build_evalset(val)
    
    score = eval_ensemble(i, train_data, data_info, users_val, movies_val, ratings_val, users_test, movies_test)
    print(score)
    scores.append(score)

NameError: name 'eval_ensemble' is not defined

In [6]:
def eval_ensemble(split, train_data, data_info, users_val, movies_val, ratings_val, users_test, movies_test):
    emb_sizes = [2,3,4,5,6,7,8,9,10,11,16,32,58]
    regs = [0.1, 0.5, 1, 5, 10]
    alphas = [0.01, 0.1, 1, 5, 8, 10, 100]
    david_val = np.zeros((len(emb_sizes) * len(regs) * len(alphas), len(users_val)))
    david_test = np.zeros((len(emb_sizes) * len(regs) * len(alphas), len(users_test)))

    for i, emb_size in enumerate(emb_sizes):
        for j, reg in enumerate(regs):
            for k, alpha in enumerate(alphas):
                giorgio = ALS(task="rating", data_info=data_info, embed_size=emb_size, n_epochs=5,
                            reg=reg, alpha=alpha, seed=42)
                giorgio.fit(train_data, verbose=0, use_cg=False, n_threads=8, metrics=["rmse", "mae", "r2"])
                david_val[len(regs)*len(alphas)*i+len(alphas)*j+k] = np.array(giorgio.predict(user=users_val, item=movies_val))
                david_test[len(regs)*len(alphas)*i+len(alphas)*j+k] = np.array(giorgio.predict(user=users_test, item=movies_test))

    yhat_val = david_val.mean(axis=0)
    yhat_test = david_test.mean(axis=0)
    save_predictions_from_pandas(f'../results_ensemble/ALS/ALS_split_{split}_val_results.csv', yhat_val, val_pd)
    save_predictions(f'../results_ensemble/ALS/ALS_split_{split}_test_results.csv', yhat_test)
    
    return mean_squared_error(yhat_val, ratings_val, squared=False)

In [9]:
# --------- generate submission -------
emb_sizes = [2,3,4,5,6,7,8,9,10,11,16,32,58]
regs = [0.1, 0.5, 1, 5, 10]
alphas = [0.01, 0.1, 1, 5, 8, 10, 100]
david = np.zeros((len(emb_sizes) * len(regs) * len(alphas), len(users_test)))

train_pd = pd.read_csv(f'../data/data_train.csv')
users_train, movies_train, ratings_train = extract_users_movies_ratings_lists(train_pd)
train = pd.DataFrame({'user': users_train, 'item': movies_train, 'label': ratings_train})
train_data, data_info = DatasetPure.build_trainset(train)


for i, emb_size in enumerate(emb_sizes):
    for j, reg in enumerate(regs):
        for k, alpha in enumerate(alphas):
            giorgio = ALS(task="rating", data_info=data_info, embed_size=emb_size, n_epochs=5,
                        reg=reg, alpha=alpha, seed=42)
            giorgio.fit(train_data, verbose=2, use_cg=False, n_threads=8, metrics=["rmse", "mae", "r2"])
            david[len(regs)*len(alphas)*i+len(alphas)*j+k] = np.array(giorgio.predict(user=users_test, item=movies_test))
            
yhat = david.mean(axis=0)
save_predictions(f'../results_ensemble/ALS/ALS-predictedSubmission.csv', yhat)

Training start time: 2022-07-19 16:34:10
Epoch 1 elapsed: 0.069s
Epoch 2 elapsed: 0.041s
Epoch 3 elapsed: 0.042s
Epoch 4 elapsed: 0.039s
Epoch 5 elapsed: 0.046s
Training start time: 2022-07-19 16:34:11
Epoch 1 elapsed: 0.036s
Epoch 2 elapsed: 0.035s
Epoch 3 elapsed: 0.035s
Epoch 4 elapsed: 0.036s
Epoch 5 elapsed: 0.035s
Training start time: 2022-07-19 16:34:13
Epoch 1 elapsed: 0.041s
Epoch 2 elapsed: 0.036s
Epoch 3 elapsed: 0.039s
Epoch 4 elapsed: 0.038s
Epoch 5 elapsed: 0.046s
Training start time: 2022-07-19 16:34:14
Epoch 1 elapsed: 0.044s
Epoch 2 elapsed: 0.038s
Epoch 3 elapsed: 0.042s
Epoch 4 elapsed: 0.040s
Epoch 5 elapsed: 0.035s
Training start time: 2022-07-19 16:34:15
Epoch 1 elapsed: 0.037s
Epoch 2 elapsed: 0.036s
Epoch 3 elapsed: 0.036s
Epoch 4 elapsed: 0.036s
Epoch 5 elapsed: 0.036s
Training start time: 2022-07-19 16:34:17
Epoch 1 elapsed: 0.036s
Epoch 2 elapsed: 0.038s
Epoch 3 elapsed: 0.035s
Epoch 4 elapsed: 0.035s
Epoch 5 elapsed: 0.036s
Training start time: 2022-07-19 16

Epoch 5 elapsed: 0.038s
Training start time: 2022-07-19 16:34:42
Epoch 1 elapsed: 0.035s
Epoch 2 elapsed: 0.035s
Epoch 3 elapsed: 0.035s
Epoch 4 elapsed: 0.035s
Epoch 5 elapsed: 0.036s
Training start time: 2022-07-19 16:34:43
Epoch 1 elapsed: 0.050s
Epoch 2 elapsed: 0.048s
Epoch 3 elapsed: 0.037s
Epoch 4 elapsed: 0.047s
Epoch 5 elapsed: 0.044s
Training start time: 2022-07-19 16:34:44
Epoch 1 elapsed: 0.036s
Epoch 2 elapsed: 0.039s
Epoch 3 elapsed: 0.042s
Epoch 4 elapsed: 0.050s
Epoch 5 elapsed: 0.041s
Training start time: 2022-07-19 16:34:46
Epoch 1 elapsed: 0.034s
Epoch 2 elapsed: 0.036s
Epoch 3 elapsed: 0.035s
Epoch 4 elapsed: 0.035s
Epoch 5 elapsed: 0.036s
Training start time: 2022-07-19 16:34:47
Epoch 1 elapsed: 0.045s
Epoch 2 elapsed: 0.040s
Epoch 3 elapsed: 0.042s
Epoch 4 elapsed: 0.041s
Epoch 5 elapsed: 0.039s
Training start time: 2022-07-19 16:34:48
Epoch 1 elapsed: 0.035s
Epoch 2 elapsed: 0.036s
Epoch 3 elapsed: 0.035s
Epoch 4 elapsed: 0.035s
Epoch 5 elapsed: 0.035s
Training s

Epoch 4 elapsed: 0.051s
Epoch 5 elapsed: 0.049s
Training start time: 2022-07-19 16:35:15
Epoch 1 elapsed: 0.052s
Epoch 2 elapsed: 0.050s
Epoch 3 elapsed: 0.051s
Epoch 4 elapsed: 0.050s
Epoch 5 elapsed: 0.053s
Training start time: 2022-07-19 16:35:16
Epoch 1 elapsed: 0.056s
Epoch 2 elapsed: 0.052s
Epoch 3 elapsed: 0.049s
Epoch 4 elapsed: 0.052s
Epoch 5 elapsed: 0.059s
Training start time: 2022-07-19 16:35:18
Epoch 1 elapsed: 0.054s
Epoch 2 elapsed: 0.051s
Epoch 3 elapsed: 0.053s
Epoch 4 elapsed: 0.050s
Epoch 5 elapsed: 0.057s
Training start time: 2022-07-19 16:35:19
Epoch 1 elapsed: 0.049s
Epoch 2 elapsed: 0.050s
Epoch 3 elapsed: 0.049s
Epoch 4 elapsed: 0.050s
Epoch 5 elapsed: 0.048s
Training start time: 2022-07-19 16:35:20
Epoch 1 elapsed: 0.052s
Epoch 2 elapsed: 0.052s
Epoch 3 elapsed: 0.049s
Epoch 4 elapsed: 0.050s
Epoch 5 elapsed: 0.050s
Training start time: 2022-07-19 16:35:22
Epoch 1 elapsed: 0.049s
Epoch 2 elapsed: 0.050s
Epoch 3 elapsed: 0.051s
Epoch 4 elapsed: 0.050s
Epoch 5 el

Training start time: 2022-07-19 16:35:47
Epoch 1 elapsed: 0.061s
Epoch 2 elapsed: 0.059s
Epoch 3 elapsed: 0.058s
Epoch 4 elapsed: 0.061s
Epoch 5 elapsed: 0.081s
Training start time: 2022-07-19 16:35:48
Epoch 1 elapsed: 0.070s
Epoch 2 elapsed: 0.057s
Epoch 3 elapsed: 0.060s
Epoch 4 elapsed: 0.060s
Epoch 5 elapsed: 0.063s
Training start time: 2022-07-19 16:35:50
Epoch 1 elapsed: 0.059s
Epoch 2 elapsed: 0.058s
Epoch 3 elapsed: 0.059s
Epoch 4 elapsed: 0.058s
Epoch 5 elapsed: 0.061s
Training start time: 2022-07-19 16:35:51
Epoch 1 elapsed: 0.059s
Epoch 2 elapsed: 0.058s
Epoch 3 elapsed: 0.061s
Epoch 4 elapsed: 0.058s
Epoch 5 elapsed: 0.060s
Training start time: 2022-07-19 16:35:52
Epoch 1 elapsed: 0.059s
Epoch 2 elapsed: 0.058s
Epoch 3 elapsed: 0.057s
Epoch 4 elapsed: 0.063s
Epoch 5 elapsed: 0.066s
Training start time: 2022-07-19 16:35:54
Epoch 1 elapsed: 0.057s
Epoch 2 elapsed: 0.059s
Epoch 3 elapsed: 0.057s
Epoch 4 elapsed: 0.062s
Epoch 5 elapsed: 0.058s
Training start time: 2022-07-19 16

Epoch 4 elapsed: 0.061s
Epoch 5 elapsed: 0.058s
Training start time: 2022-07-19 16:36:21
Epoch 1 elapsed: 0.061s
Epoch 2 elapsed: 0.059s
Epoch 3 elapsed: 0.061s
Epoch 4 elapsed: 0.059s
Epoch 5 elapsed: 0.060s
Training start time: 2022-07-19 16:36:22
Epoch 1 elapsed: 0.057s
Epoch 2 elapsed: 0.067s
Epoch 3 elapsed: 0.078s
Epoch 4 elapsed: 0.057s
Epoch 5 elapsed: 0.066s
Training start time: 2022-07-19 16:36:24
Epoch 1 elapsed: 0.091s
Epoch 2 elapsed: 0.071s
Epoch 3 elapsed: 0.089s
Epoch 4 elapsed: 0.071s
Epoch 5 elapsed: 0.073s
Training start time: 2022-07-19 16:36:25
Epoch 1 elapsed: 0.075s
Epoch 2 elapsed: 0.078s
Epoch 3 elapsed: 0.079s
Epoch 4 elapsed: 0.076s
Epoch 5 elapsed: 0.076s
Training start time: 2022-07-19 16:36:27
Epoch 1 elapsed: 0.084s
Epoch 2 elapsed: 0.071s
Epoch 3 elapsed: 0.087s
Epoch 4 elapsed: 0.075s
Epoch 5 elapsed: 0.072s
Training start time: 2022-07-19 16:36:28
Epoch 1 elapsed: 0.072s
Epoch 2 elapsed: 0.078s
Epoch 3 elapsed: 0.073s
Epoch 4 elapsed: 0.079s
Epoch 5 el

...

Epoch 4 elapsed: 0.161s
Epoch 5 elapsed: 0.164s
Training start time: 2022-07-19 16:42:38
Epoch 1 elapsed: 0.162s
Epoch 2 elapsed: 0.162s
Epoch 3 elapsed: 0.159s
Epoch 4 elapsed: 0.165s
Epoch 5 elapsed: 0.162s
Training start time: 2022-07-19 16:42:40
Epoch 1 elapsed: 0.165s
Epoch 2 elapsed: 0.167s
Epoch 3 elapsed: 0.189s
Epoch 4 elapsed: 0.169s
Epoch 5 elapsed: 0.163s
Training start time: 2022-07-19 16:42:42
Epoch 1 elapsed: 0.159s
Epoch 2 elapsed: 0.157s
Epoch 3 elapsed: 0.158s
Epoch 4 elapsed: 0.160s
Epoch 5 elapsed: 0.165s
Training start time: 2022-07-19 16:42:44
Epoch 1 elapsed: 0.164s
Epoch 2 elapsed: 0.180s
Epoch 3 elapsed: 0.164s
Epoch 4 elapsed: 0.160s
Epoch 5 elapsed: 0.163s
Training start time: 2022-07-19 16:42:46
Epoch 1 elapsed: 0.167s
Epoch 2 elapsed: 0.164s
Epoch 3 elapsed: 0.168s
Epoch 4 elapsed: 0.165s
Epoch 5 elapsed: 0.163s
Training start time: 2022-07-19 16:42:48
Epoch 1 elapsed: 0.194s
Epoch 2 elapsed: 0.164s
Epoch 3 elapsed: 0.172s
Epoch 4 elapsed: 0.273s
Epoch 5 el

Training start time: 2022-07-19 16:43:29
Epoch 1 elapsed: 0.205s
Epoch 2 elapsed: 0.209s
Epoch 3 elapsed: 0.202s
Epoch 4 elapsed: 0.221s
Epoch 5 elapsed: 0.217s
Training start time: 2022-07-19 16:43:31
Epoch 1 elapsed: 0.200s
Epoch 2 elapsed: 0.200s
Epoch 3 elapsed: 0.227s
Epoch 4 elapsed: 0.210s
Epoch 5 elapsed: 0.212s
Training start time: 2022-07-19 16:43:33
Epoch 1 elapsed: 0.200s
Epoch 2 elapsed: 0.205s
Epoch 3 elapsed: 0.223s
Epoch 4 elapsed: 0.227s
Epoch 5 elapsed: 0.227s
Training start time: 2022-07-19 16:43:35
Epoch 1 elapsed: 0.210s
Epoch 2 elapsed: 0.204s
Epoch 3 elapsed: 0.234s
Epoch 4 elapsed: 0.232s
Epoch 5 elapsed: 0.222s
Training start time: 2022-07-19 16:43:38
Epoch 1 elapsed: 0.215s
Epoch 2 elapsed: 0.210s
Epoch 3 elapsed: 0.213s
Epoch 4 elapsed: 0.215s
Epoch 5 elapsed: 0.232s
Training start time: 2022-07-19 16:43:40
Epoch 1 elapsed: 0.191s
Epoch 2 elapsed: 0.208s
Epoch 3 elapsed: 0.209s
Epoch 4 elapsed: 0.213s
Epoch 5 elapsed: 0.223s
Training start time: 2022-07-19 16

Epoch 2 elapsed: 0.221s
Epoch 3 elapsed: 0.202s
Epoch 4 elapsed: 0.226s
Epoch 5 elapsed: 0.220s
Training start time: 2022-07-19 16:44:25
Epoch 1 elapsed: 0.229s
Epoch 2 elapsed: 0.215s
Epoch 3 elapsed: 0.209s
Epoch 4 elapsed: 0.228s
Epoch 5 elapsed: 0.229s
Training start time: 2022-07-19 16:44:27
Epoch 1 elapsed: 0.214s
Epoch 2 elapsed: 0.205s
Epoch 3 elapsed: 0.225s
Epoch 4 elapsed: 0.219s
Epoch 5 elapsed: 0.216s
Training start time: 2022-07-19 16:44:29
Epoch 1 elapsed: 0.221s
Epoch 2 elapsed: 0.204s
Epoch 3 elapsed: 0.214s
Epoch 4 elapsed: 0.221s
Epoch 5 elapsed: 0.233s
Training start time: 2022-07-19 16:44:31
Epoch 1 elapsed: 0.409s
Epoch 2 elapsed: 0.400s
Epoch 3 elapsed: 0.430s
Epoch 4 elapsed: 0.439s
Epoch 5 elapsed: 0.472s
Training start time: 2022-07-19 16:44:35
Epoch 1 elapsed: 0.419s
Epoch 2 elapsed: 0.414s
Epoch 3 elapsed: 0.457s
Epoch 4 elapsed: 0.416s
Epoch 5 elapsed: 0.439s
Training start time: 2022-07-19 16:44:38
Epoch 1 elapsed: 0.390s
Epoch 2 elapsed: 0.440s
Epoch 3 el

Epoch 4 elapsed: 0.434s
Epoch 5 elapsed: 0.425s
Training start time: 2022-07-19 16:45:45
Epoch 1 elapsed: 0.424s
Epoch 2 elapsed: 0.457s
Epoch 3 elapsed: 0.464s
Epoch 4 elapsed: 0.467s
Epoch 5 elapsed: 0.454s
Training start time: 2022-07-19 16:45:48
Epoch 1 elapsed: 0.457s
Epoch 2 elapsed: 0.463s
Epoch 3 elapsed: 0.440s
Epoch 4 elapsed: 0.473s
Epoch 5 elapsed: 0.455s
Training start time: 2022-07-19 16:45:52
Epoch 1 elapsed: 0.452s
Epoch 2 elapsed: 0.448s
Epoch 3 elapsed: 0.448s
Epoch 4 elapsed: 0.408s
Epoch 5 elapsed: 0.448s
Training start time: 2022-07-19 16:45:55
Epoch 1 elapsed: 0.436s
Epoch 2 elapsed: 0.460s
Epoch 3 elapsed: 0.451s
Epoch 4 elapsed: 0.437s
Epoch 5 elapsed: 0.474s
Training start time: 2022-07-19 16:45:58
Epoch 1 elapsed: 0.439s
Epoch 2 elapsed: 0.440s
Epoch 3 elapsed: 0.459s
Epoch 4 elapsed: 0.449s
Epoch 5 elapsed: 0.501s
Training start time: 2022-07-19 16:46:02
Epoch 1 elapsed: 0.452s
Epoch 2 elapsed: 0.461s
Epoch 3 elapsed: 0.473s
Epoch 4 elapsed: 0.442s
Epoch 5 el

Training start time: 2022-07-19 16:47:39
Epoch 1 elapsed: 0.920s
Epoch 2 elapsed: 0.911s
Epoch 3 elapsed: 0.992s
Epoch 4 elapsed: 0.908s
Epoch 5 elapsed: 0.921s
Training start time: 2022-07-19 16:47:45
Epoch 1 elapsed: 0.902s
Epoch 2 elapsed: 0.908s
Epoch 3 elapsed: 0.932s
Epoch 4 elapsed: 0.998s
Epoch 5 elapsed: 0.923s
Training start time: 2022-07-19 16:47:51
Epoch 1 elapsed: 0.888s
Epoch 2 elapsed: 0.897s
Epoch 3 elapsed: 0.898s
Epoch 4 elapsed: 0.915s
Epoch 5 elapsed: 0.953s
Training start time: 2022-07-19 16:47:57
Epoch 1 elapsed: 0.875s
Epoch 2 elapsed: 0.906s
Epoch 3 elapsed: 0.914s
Epoch 4 elapsed: 0.894s
Epoch 5 elapsed: 0.905s
Training start time: 2022-07-19 16:48:03
Epoch 1 elapsed: 0.879s
Epoch 2 elapsed: 0.885s
Epoch 3 elapsed: 0.923s
Epoch 4 elapsed: 0.901s
Epoch 5 elapsed: 0.899s
Training start time: 2022-07-19 16:48:09
Epoch 1 elapsed: 0.863s
Epoch 2 elapsed: 0.912s
Epoch 3 elapsed: 0.911s
Epoch 4 elapsed: 0.902s
Epoch 5 elapsed: 0.904s
Training start time: 2022-07-19 16